# 1. Data Collection and Cleaning

The purpose of this notebook is to clean, simplify, and save various datasets so that the data can be merged and visualized.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import random
import zipfile

In [2]:
random.seed(6)

## Develop API to match coordinates to census tract

In [3]:
def coord_to_tract(lat, long):
    """ This function identifies the FIPs census tract number for a given latitude and longitude using the 2010 census.
    
    args:
        lat (float): Latitude
        long (float): Longitude
        
    return:
        tract (int): census tract number, which is the first 11 digits of the census block FIPs
    """
    url = 'https://geo.fcc.gov/api/census/block/find'
    
    params = {
        'latitude': lat,
        'longitude': long,
        'censusYear': 2010,
        'format':'json'
    }
    res = requests.get(url,params)
    if res.status_code == 200:
        fips = res.json()['Block']['FIPS']
        return fips[0:11]
    else:
        print('request_failed')

## Datasets

### US Petroleum Refineries

In [4]:
# Read in data, rename columns, and filter to Texas only
df_refineries = pd.read_csv('../datasets/us-petroleum-refineries.csv')

df_refineries.columns = df_refineries.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

df_refineries = df_refineries[df_refineries['state'] == 'Texas']

In [5]:
# Obtain tract number
refinery_tract = []
for refinery in df_refineries.index:
    coords = df_refineries.loc[refinery,['latitude','longitude']]
    refinery_tract.append(coord_to_tract(coords[0],coords[1]))

df_refineries['tract'] = refinery_tract

In [6]:
# Reduce columns
df_refineries = df_refineries[['company', 'corp', 'tract']]

In [7]:
df_refineries['tract'].value_counts() # there are duplicate refineries in tracts

48355006300    3
48167726200    3
48245011201    1
48297950100    1
48201233703    1
48201252600    1
48493000500    1
48199031000    1
48245011600    1
48201324200    1
48355000600    1
48201320500    1
48245006600    1
48201343600    1
48341950200    1
48227950100    1
48039662700    1
48233951000    1
48245000700    1
48201253700    1
48029192200    1
48201324100    1
48423000600    1
48141003501    1
48201252500    1
Name: tract, dtype: int64

In [8]:
# Let's combine refineries to 1 tract number and add a number of refineries to each tract

df_refineries['number_refineries'] = 1

# For the repeats, let's combine refinery company names and corporation names
for i in df_refineries['tract'].value_counts()[0:2].index:
    matches = df_refineries[(df_refineries['tract'] == i)]
    company_names = matches['company'].str.cat(sep=', ')
    corp_names = matches['corp'].str.cat(sep=', ')
    df_refineries.loc[matches.index[0],['company','corp','number_refineries']] = [company_names, corp_names, matches.shape[0]]
df_refineries.drop_duplicates(subset='tract',inplace=True)
df_refineries.set_index('tract',inplace=True)

### EPA 2017 AirToxScreen: Cancer Risk by Source Group

In [9]:
# Read in data, rename columns, and filter to Texas only
df_epa_cancer_by_source = pd.read_excel('../datasets/national_cancerrisk_by_tract_srcgrp.xlsx')

df_epa_cancer_by_source.columns = df_epa_cancer_by_source.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

df_epa_cancer_by_source_TX = df_epa_cancer_by_source[df_epa_cancer_by_source['state'] == 'TX']

In [10]:
# Reduce columns
df_epa_cancer_by_source_TX = df_epa_cancer_by_source_TX[['tract','total_cancer_risk_(per_million)','pt-stationarypoint_cancer_risk_(per_million)']]

In [11]:
df_epa_cancer_by_source_TX.set_index('tract',inplace=True)

### EPA 2017 AirToxScreen: Cancer Risk by Pollutant

In [12]:
# Read in data, rename columns, and filter to Texas only
df_epa_cancer_by_pollutant = pd.read_excel('../datasets/national_cancerrisk_by_tract_poll.xlsx',)

df_epa_cancer_by_pollutant.columns = df_epa_cancer_by_pollutant.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

df_epa_cancer_by_pollutant_TX = df_epa_cancer_by_pollutant[df_epa_cancer_by_pollutant['state'] == 'TX']

In [13]:
# Reduce columns
df_epa_cancer_by_pollutant_TX = df_epa_cancer_by_pollutant_TX[['tract', 'benzene',
'coke_oven_emissions',
'1,3-butadiene']]

In [14]:
df_epa_cancer_by_pollutant_TX.set_index('tract',inplace=True)

In [15]:
df_epa_cancer_by_pollutant_TX.columns = df_epa_cancer_by_pollutant_TX.columns + '_cancer_risk_(per_million)'

### EPA 2017 AirToxScreen: Emissions by Source

In [16]:
# Read in data, rename columns, and filter to Texas only
df_epa_emissions_by_source = pd.read_excel('../datasets/point_fac_2017_emissions.xlsx')

df_epa_emissions_by_source.columns = df_epa_emissions_by_source.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

df_epa_emissions_by_source_TX = df_epa_emissions_by_source[df_epa_emissions_by_source['state'] == 'Texas']

df_epa_emissions_by_source_TX.loc[:,'state'] = 'TX'

# Drop non-numerical values for facility-id
df_epa_emissions_by_source_TX['facility_id'] = pd.to_numeric(df_epa_emissions_by_source_TX['facility_id'], errors='coerce')

df_epa_emissions_by_source_TX.dropna(inplace=True)

/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/3583804125.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epa_emissions_by_source_TX.loc[:,'state'] = 'TX'
/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/3583804125.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epa_emissions_by_source_TX['facility_id'] = pd.to_numeric(df_epa_emissions_by_source_TX['facility_id'], errors='coerce')
/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/3583804125.py:

In [17]:
# Obtain tract number
emission_tract = []
for source in df_epa_emissions_by_source_TX.index:
    coords = df_epa_emissions_by_source_TX.loc[source,['latitude','longitude']]
    emission_tract.append(coord_to_tract(coords[0],coords[1]))

df_epa_emissions_by_source_TX['tract'] = emission_tract

/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/3200267812.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epa_emissions_by_source_TX['tract'] = emission_tract


In [18]:
# Reduce columns and groupby to combine by tract number
df_epa_emissions_by_source_TX = df_epa_emissions_by_source_TX[['tract','benzene_(year_2017_tons)',
        'toluene_(year_2017_tons)','ethyl_benzene_(year_2017_tons)',
        'xylenes_(mixed_isomers)_(year_2017_tons)','diesel_pm_(year_2017_tons)',
        '2,2,4-trimethylpentane_(year_2017_tons)','coke_oven_emissions_(year_2017_tons)',
        '1,3-butadiene_(year_2017_tons)']
                            ]

df_epa_emissions_by_source_TX = df_epa_emissions_by_source_TX.groupby('tract').sum()

In [19]:
df_epa_emissions_by_source_TX

,benzene_(year_2017_tons),toluene_(year_2017_tons),ethyl_benzene_(year_2017_tons),xylenes_(mixed_isomers)_(year_2017_tons),diesel_pm_(year_2017_tons),"2,2,4-trimethylpentane_(year_2017_tons)",coke_oven_emissions_(year_2017_tons),"1,3-butadiene_(year_2017_tons)"
tract,,,,,,,,
48001950100,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
48001950600,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
48001951000,0.000032,0.328400,2.712000,14.756400,0.0,0.000000,0.0,0.445400
48001951100,0.153100,0.107005,0.011852,0.045225,0.0,0.014812,0.0,0.068208
48003950100,3.050000,1.427061,0.535908,2.310171,0.0,0.664438,0.0,1.360713
...,...,...,...,...,...,...,...,...
48501950200,0.261663,1.422007,0.273398,0.529271,0.0,0.048500,0.0,0.117893
48503950400,0.000838,0.001357,0.000000,0.000017,0.0,0.000000,0.0,0.000000
48503950600,0.000000,1.525500,0.000000,0.816000,0.0,0.000000,0.0,0.000000


### EPA 2017 AirToxScreen: All Hazard Indices by Source

In [20]:
# Read in data, rename columns, and filter to Texas only
df_epa_all_hazard_index = pd.read_excel('../datasets/national_allhi_by_tract.xlsx')

df_epa_all_hazard_index.columns = df_epa_all_hazard_index.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

df_epa_all_hazard_index_TX = df_epa_all_hazard_index[df_epa_all_hazard_index['state'] == 'TX']


In [21]:
# Reduce columns
df_epa_all_hazard_index_TX = df_epa_all_hazard_index_TX[['tract','county','population','respiratory_hi', 'neurological_hi','developmental_hi',
       'reproductive_hi', 'kidney_hi','immunological_hi','whole_body_hi']]

In [22]:
df_epa_all_hazard_index_TX.set_index('tract',inplace=True)

### Combine Datasets

In [23]:
df_merged = pd.merge(df_refineries.set_index(df_refineries.index.astype(int)),df_epa_all_hazard_index_TX,how = 'outer',left_index=True, right_index=True)

In [24]:
df_merged['number_refineries'].fillna(0,inplace=True)

In [25]:
df_merged.fillna('No refineries',inplace=True)

In [26]:
df_merged = pd.merge(df_merged,df_epa_cancer_by_source_TX,how = 'outer',left_index=True, right_index=True)

In [27]:
df_merged = pd.merge(df_merged,df_epa_cancer_by_pollutant_TX,how = 'outer',left_index=True, right_index=True)

In [28]:
df_merged = pd.merge(df_merged,df_epa_emissions_by_source_TX.set_index(df_epa_emissions_by_source_TX.index.astype(int)),how='left',left_index=True, right_index=True)

In [29]:
df_merged.groupby('company')[['total_cancer_risk_(per_million)','number_refineries']].mean().astype(int).sort_values(by='total_cancer_risk_(per_million)',ascending=False)

,total_cancer_risk_(per_million),number_refineries
company,,
EQUISTAR CHEMICALS LP,80,1
PETROMAX REFINING CO LLC,60,1
KINDER MORGAN CRUDE & CONDENSATE,50,1
VALERO REFINING CO TEXAS LP,50,1
TOTAL PETROCHEMICALS & REFINING USA,50,1
PASADENA REFINING SYSTEMS INC,50,1
PREMCOR REFINING GROUP INC,50,1
DEER PARK REFINING LTD PARTNERSHIP,50,1
WESTERN REFINING COMPANY LP,50,1


In [30]:
# save to dataset
df_merged.to_csv('../cleaned_datasets/combined_texas_refinery_risks_emissions.csv')

### Texas Health Data by Census Tract

In [31]:
# Read in data and drop unncessary columns
df_health_data = pd.read_csv('../datasets/Texas_Health_Data.csv')

df_health_data.drop(columns='Unnamed: 0', inplace=True)

df_health_data.drop(
    columns= ['StateDesc','CountyName', 'Category','Data_Value_Footnote',
              'CategoryID', 'MeasureId','DataValueTypeID','Short_Question_Text', 'Data_Value_Footnote_Symbol'], inplace=True)


In [32]:
# Save cancer data to separate file
df_cancer = df_health_data[df_health_data['Measure']=='Cancer (excluding skin cancer) among adults aged >=18 years']

df_cancer.to_csv('../cleaned_datasets/cancer_rates.csv')

In [33]:
# Save poor self-rated health data to separate file
df_poor_health = df_health_data[df_health_data['Measure']=='Fair or poor self-rated health status among adults aged >=18 years']

df_poor_health.to_csv('../cleaned_datasets/poor_health_rates.csv')

### Toxic Release Inventory

In [34]:
# Read in data, reduce size, rename columns, and eliminate duplicate facilities
tri = pd.read_csv('../datasets/tri_2017_tx.csv')

tri = tri[['4. FACILITY NAME','12. LATITUDE', '13. LONGITUDE','15. PARENT CO NAME','20. INDUSTRY SECTOR']]

tri.columns = [x.split('. ')[1].lower().replace(' ','_') for x in tri.columns]

tri = tri.drop_duplicates(subset=['facility_name'])

In [35]:
# Obtain tract number
tri_tract = []
for facility in tri.index:
    coords = tri.loc[facility,['latitude','longitude']]
    tri_tract.append(coord_to_tract(coords[0],coords[1]))
        

In [56]:
tri['tract'] = tri_tract

In [57]:
# Reduce columns further
tri = tri[['facility_name', 'industry_sector', 'tract']]

In [58]:
# Let's combine sites to 1 tract number and add a number of sites to each tract

tri['number_facilities'] = 1


# For the repeats, let's combine facility names, company names, and industry

repeats = list(tri['tract'].value_counts().items())

repeat_tracts = [x[0] for x in repeats if x[1] > 1]

for i in repeat_tracts:
    matches = tri[(tri['tract'] == i)]
    facility_names = matches['facility_name'].str.cat(sep=', ')
    industry_names = ', '.join(x for x in set(matches['industry_sector']))
    tri.loc[matches.index[0],['facility_name','industry_sector','number_facilities']] = [facility_names, industry_names, matches.shape[0]]

tri.drop_duplicates(subset='tract',inplace=True)

tri.set_index('tract',inplace=True)

In [60]:
tri.to_csv('../cleaned_datasets/tri_2017_tx_cleaned.csv')

### ACS SNAP

In [61]:
with zipfile.ZipFile("../datasets/ACSST5Y2019.S1701_2022-05-06T144748.zip","r") as zip_ref:
    zip_ref.extractall("../cleaned_datasets/ACSPoverty")
    
with zipfile.ZipFile('../datasets/ACSST5Y2019.S2201_2022-05-06T144719.zip','r') as zip_r:
    zip_r.extractall('../cleaned_datasets/ACSpovertySNAP')

In [62]:
# Read in data
df_snap = pd.read_csv('../cleaned_datasets/ACSpovertySNAP/ACSST5Y2019.S2201_data_with_overlays_2022-05-06T144356.csv')

df_metasnap = pd.read_csv('../cleaned_datasets/ACSpovertySNAP/ACSST5Y2019.S2201_metadata_2022-05-06T144356.csv')

/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/3769334036.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,70,71,72,73,74,75,76,77,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,226,227,228,229,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,374,375,376,377,378,379,380,381) have mixed types. Specify dtype option on import or set low_memory=False.
  df_snap = pd

In [63]:
df_snapped = df_snap.loc[:,~df_snap.columns.str.endswith('M')] #no margin of errors!

In [64]:
# Rename columns with metadata
snap_dict = {df_metasnap.iloc[i,0]:df_metasnap.iloc[i,1] for i in list(range(len(df_metasnap.iloc[:,0])))}

snap_dict['GEO_ID'] = 'id' #the header, can't take that along, 
# could change that in the reading phase, but this should be fine

df_snapped.rename(columns=snap_dict,inplace=True)

/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/4008989000.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_snapped.rename(columns=snap_dict,inplace=True)


In [65]:
# Filter Texas only data and reduce columns
df_snaptexas = df_snapped[df_snapped['Geographic Area Name'].str.endswith('Texas')]

df_snaptxshort = df_snaptexas[['id','Geographic Area Name','Estimate!!Total!!Households',
                               'Estimate!!Total!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level',
                              'Estimate!!Households receiving food stamps/SNAP!!Households',
                              'Estimate!!Percent!!WORK STATUS!!Families!!No workers in past 12 months'
                              ]]

df_snaptxshort.set_axis(axis=1,labels=['id','geoid','households','below_poverty','hh_snap','hh_nowork'],inplace=True)

In [66]:
# Obtain tract num
df_snaptxshort['id']= [y for x,y in df_snaptxshort['id'].str.split('US')]

/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/620470002.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_snaptxshort['id']= [y for x,y in df_snaptxshort['id'].str.split('US')]


In [67]:
# Save to csv
df_snaptxshort.to_csv('../cleaned_datasets/SNAPinfoTX.csv',index=False)

### ACS Poverty

In [68]:
# Read in data
df_pov = pd.read_csv('../cleaned_datasets/ACSPoverty/ACSST5Y2019.S1701_data_with_overlays_2022-05-06T144518.csv')

df_metapov = pd.read_csv('../cleaned_datasets/ACSPoverty/ACSST5Y2019.S1701_metadata_2022-05-06T144518.csv')

/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/3462979621.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,240,241,242,243,244,245) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pov = pd.read_csv('../cle

In [69]:
# Filter Texas only
df_povtexas = df_pov[df_pov['NAME'].str.endswith('Texas')]

In [70]:
df_pov = df_pov.loc[:,~df_pov.columns.str.endswith('M')]

In [71]:
# Rename columns with metadata
pov_dict = {df_metapov.iloc[i,0]:df_metapov.iloc[i,1] for i in list(range(len(df_metapov.iloc[:,0])))}

pov_dict['GEO_ID']= 'id'

df_povtexas.rename(columns=pov_dict,inplace=True)

In [72]:
# Reduce columns
df_povtxshort = df_povtexas[['id','Geographic Area Name',
                            'Estimate!!Total!!Population for whom poverty status is determined',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!50 percent of poverty level',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!125 percent of poverty level',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!150 percent of poverty level',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!185 percent of poverty level',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!200 percent of poverty level',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!300 percent of poverty level',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!400 percent of poverty level',
                            'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!500 percent of poverty level']]

df_povtxshort.set_axis(axis=1,labels=['id','geoid','population','below50pov','below125pov','below150pov','below185pov','below200pov',
                                     'below300pov','below400pov','below500pov'],inplace=True)








In [73]:
# Obtain tract num
df_povtxshort['id']= [y for x,y in df_povtxshort['id'].str.split('US')]

/var/folders/0z/k5klxhhj5j9g8hc5ygn0nwsr0000gn/T/ipykernel_12720/359515212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_povtxshort['id']= [y for x,y in df_povtxshort['id'].str.split('US')]


In [74]:
# Save to csv
df_povtxshort.to_csv('../cleaned_datasets/PovertyinfoTX.csv',index=False)